# Open Space Organizer

We want to create a program that assigns 24 people to 6 tables in an openspace. Before getting started, take inventory what do we need:

- People
- Seats & Tables
- An OpenSpace

It's a good practice to start simple while you grasp the logic of the program you are trying to build and test often. For us this can translate to,

- People -> List of Names (later we can figure out how to use a file)
- Seats & Tables -> Class
- An OpenSpace -> Class

Below I've created a list of your new colleagues for reference!

In [5]:
new_collegues = ["Aleksei","Amine","Anna","Astha","Brigitta",
                 "Bryan","Ena","Esra","Faranges","Frédéric",
                 "Hamideh","Héloïse","Imran","Intan K.",
                 "Jens","Kristin","Michiel","Nancy","Pierrick",
                 "Sandrine","Tim","Viktor","Welederufeal","Živile"]
len(new_collegues), new_collegues[:5]

(24, ['Aleksei', 'Amine', 'Anna', 'Astha', 'Brigitta'])

## Step 1: Build a Seat

Create a class called `Seat` with two attributes:

- `free` which is a boolean.
- `occupant` which is a string.

and 2 functions : 

- `set_occupant(name)` which allows the program to assign someone a seat if it's free
- `remove_occupant()` which  remove someone from a seat and return the name of the person occupying the seat before


In [1]:
class Seat:
	"""
	Represents a single seat.

	Attributes:
		free (bool): Whether the seat is free.
		occupant (str): Name of the person occupying the seat, empty string if free.
	"""

	def __init__(self) -> None:
		"""Initialize seat as free with no occupant."""
		self.free: bool = True
		self.occupant: str = ""

	def set_occupant(self, name: str) -> bool:
		"""Assign a person to the seat if it's free.

		Args:
			name (str): The person's name.

		Returns:
			bool: True if assignment succeeded, False if already occupied.
		"""
		if self.free:
			self.occupant = name
			self.free = False
			return True
		return False

	def remove_occupant(self) -> str:
		"""Remove and return the current occupant (or empty string if free)."""
		if not self.free:
			name = self.occupant
			self.occupant = ""
			self.free = True
			return name
		return ""

	def __str__(self) -> str:
		"""Human-readable status of the seat."""
		return "Seat is free" if self.free else f"Seat occupied by {self.occupant}"

In [2]:
# Test your code (assign yourself you a Seat)
my_seat = Seat()
print("Before:", my_seat)
assigned = my_seat.set_occupant("Amine")
print("Assigned?", assigned, "| After:", my_seat)
removed = my_seat.remove_occupant()
print("Removed:", removed, "| After:", my_seat)

Before: Seat is free
Assigned? True | After: Seat occupied by Amine
Removed: Amine | After: Seat is free


What is the input and the output of your Seat class? Does it make sense?

## Step 2: Build a Table

Create a class `Table` with ? attributes:

- `capacity` which is an integer
- `seats` which is a list of `Seat` objects (size = `capacity`)

and 3 functions : 
- `has_free_spot()` that returns a boolean (True if a spot is available)
- `assign_seat(name)` that places someone at the table
- `left_capacity()` that returns an integer

Question: Which attributes make sense to give? For now let's say we want to build 6 tables with 4 seats.


In [6]:
class Table:
	"""
	Represents a table with a fixed number of seats.

	Attributes:
		capacity (int): Total number of seats.
		seats (list[Seat]): Seats at this table.
	"""

	def __init__(self, capacity: int) -> None:
		"""Create a table with `capacity` new free Seat objects."""
		self.capacity: int = capacity
		self.seats: list[Seat] = [Seat() for _ in range(capacity)]

	def has_free_spot(self) -> bool:
		"""Return True if any seat is free."""
		for seat in self.seats:
			if seat.free:
				return True
		return False

	def assign_seat(self, name: str) -> bool:
		"""Assign `name` to the first free seat, return True if successful."""
		for seat in self.seats:
			if seat.set_occupant(name):
				return True
		return False

	def left_capacity(self) -> int:
		"""Return how many seats are still free."""
		count = 0
		for seat in self.seats:
			if seat.free:
				count += 1
		return count

	def __str__(self) -> str:
		"""Summary of seat availability and occupants for this table."""
		occupied = [seat.occupant for seat in self.seats if not seat.free]
		if occupied:
			return f"Table ({len(occupied)}/{self.capacity}) occupied by: {', '.join(occupied)}"
		return f"Table (0/{self.capacity}) occupied by: No one"

In [7]:
# Test your code (assign the colleagues at your table to a Table)
tbl = Table(capacity=4)
for name in new_collegues[:3]:
	assert tbl.assign_seat(name)
print(tbl)
print("Has free spot?", tbl.has_free_spot())
print("Left capacity:", tbl.left_capacity())

Table (3/4) occupied by: Aleksei, Amine, Anna
Has free spot? True
Left capacity: 1


Does the output of you test make sense? Check that each method returns the correct value.

## Step 3: Build an OpenSpace

Create a class `Openspace` that contains these attributes:

- `tables` which is a list of `Table`. _(you will need to import `Table` from `table.py`)_. 
- `number_of_tables` which is an integer.

And some methods:

- `organize(names)` that will:
  - **randomly** assign people to `Seat` objects in the different `Table` objects.
- `display()` display the different tables and there occupants in a nice and readable way
- `store(filename)` store the repartition in an file

In [8]:
import random
from utils.table import Table


class Openspace:
    """
    A class representing an open space containing multiple tables.

    Attributes:
            tables (List[Table]): The list of tables in the open space.
            number_of_tables (int): The total number of tables.
    """

    def __init__(self, number_of_tables: int, table_capacity: int) -> None:
        """
        Initialize an Openspace with a given number of tables and their seat capacity.

        Args:
                number_of_tables (int): Number of tables in the open space.
                table_capacity (int): Number of seats per table.
        """
        self.number_of_tables: int = number_of_tables
        self.tables: list[Table] = [
            Table(table_capacity) for _ in range(number_of_tables)
        ]

    def organize(self, names: list[str]) -> None:
        """
        Randomly assign people to available seats across all tables.

        Args:
                names (List[str]): The list of people to assign to seats.
        """
        random.shuffle(names)
        index = 0
        for table in self.tables:
            while table.has_free_spot() and index < len(names):
                table.assign_seat(names[index])
                index += 1
            if index >= len(names):
                break

    def display(self) -> None:
        """
        Display all 6 tables (4 seats each) in a visual rectangular layout.
        The table name appears centered inside the rectangle,
        and seats are evenly spaced around it.
        """

        print("OPEN SPACE SEATING ARRANGEMENT\n")

        for i, table in enumerate(self.tables, start=1):

            # Ensure layout only runs for 4 seats
            if len(table.seats) != 4:
                continue

            # Extract occupant names (or 'Free')
            s = [seat.occupant if not seat.free else "Free" for seat in table.seats]

            # Layout:
            #   [Seat1]          [Seat2]
            #         +-------------+
            #         |   Table i   |
            #         +-------------+
            #   [Seat3]          [Seat4]

            top_row = f"  [{s[0]:^8}]          [{s[1]:^8}]"
            table_box = f"        +-------------+\n        |  Table {i:^3}  |\n        +-------------+"
            bottom_row = f"  [{s[2]:^8}]          [{s[3]:^8}]"

            print(top_row)
            print(table_box)
            print(bottom_row)
            print("\n" + " " * 2 + "-" * 44 + "\n")

    def store(self, filename: str) -> None:
        """
        Store the current seating arrangement in a text file.

        Args:
                filename (str): The name of the file where the arrangement will be saved.
        """
        with open(filename, "w", encoding="utf-8") as file:
            for i, table in enumerate(self.tables, start=1):
                file.write(f"Table {i}:\n")
                for j, seat in enumerate(table.seats, start=1):
                    status = f"  Seat {j}: {'Free' if seat.free else seat.occupant}\n"
                    file.write(status)
                file.write("\n")

    def __str__(self) -> str:
        """
        Return a formatted string summary of the Open Space.

        Returns:
                str: Summary with tables and seat occupancy.
        """
        result = "OpenSpace Summary:\n"
        for i, table in enumerate(self.tables, start=1):
            result += f"- Table {i}: {table.left_capacity()} seats left\n"
        return result


In [9]:
# Test your code (assign everyone in the class to a table)
space = Openspace(number_of_tables=6, table_capacity=4)
space.organize(new_collegues)
space.display()

# Also test storing to file
_output_file = "notebook_output.txt"
space.store(_output_file)
with open(_output_file, "r", encoding="utf-8") as f:
	print("\nStored file preview:")
	for i, line in enumerate(f.readlines(), start=1):
		print(f"{i}: {line.rstrip()}")

OPEN SPACE SEATING ARRANGEMENT

  [  Tim   ]          [ Viktor ]
        +-------------+
        |  Table  1   |
        +-------------+
  [  Jens  ]          [ Nancy  ]

  --------------------------------------------

  [Kristin ]          [Héloïse ]
        +-------------+
        |  Table  2   |
        +-------------+
  [Intan K.]          [ Živile ]

  --------------------------------------------

  [Hamideh ]          [ Bryan  ]
        +-------------+
        |  Table  3   |
        +-------------+
  [Sandrine]          [Pierrick]

  --------------------------------------------

  [Michiel ]          [Frédéric]
        +-------------+
        |  Table  4   |
        +-------------+
  [  Esra  ]          [Faranges]

  --------------------------------------------

  [ Amine  ]          [ Imran  ]
        +-------------+
        |  Table  5   |
        +-------------+
  [Brigitta]          [  Ena   ]

  --------------------------------------------

  [Welederufeal]          [ Astha

Hurray! You have the algorithm logic working. Next steps we transform this into some scripts! **Big note:** Once you move to the scrips you may need to adapt your logic, don't fret this is normal!